In [ ]:
# Table of Contents
 <p><div class="lev1"><a href="# Step 1. Process data"><span class="toc-item-num">1&nbsp;&nbsp;</span>TODO: Following given notebooks in HW1, create table of contents here :) </a></div>

This Jupyter extension might help. https://github.com/minrk/ipython_extensions

# Homework 03 - Interactive Viz

# Overview 

### Objective : Build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows intuitively how much grant money goes to each Swiss canton.

##### What data do we have?

The SNSF (Swiss National Science Foundation) has graciously provided the [P<sup>3</sup>](http://p3.snf.ch/) database, which contains data on research projects approved by the SNSF. We will just use the [Grants Data](http://p3.snf.ch/P3Export/P3_GrantExport.csv) (saved in `data/P3_GrantExport`) as this contains the grant amounts going to different universities.

We are also given the file `data/ch-cantons.topojson.json`, which contains the geo-coordinates of each Swiss canton.

##### What needs to be done?

The Grants Data does not contain the Canton of the University/Institution that received the funding. Therefore, we will need to deduce the Canton from the University/Institution. Once this is accomplished, we can sum the grant money that was allocated to each Canton.

##### How will this be done?

We will use the following Python packages to accomplish our objective:
1. `pandas` to import, clean, and wrangle the Grants Data
2. `json` and `pprint` to read the `data/ch-cantons.topojson.json` file
3. `requests` with the [GeoNames API](http://www.geonames.org/export/web-services.html) and `googlemaps` (a [Python wrapper](https://github.com/googlemaps/google-maps-services-python) for Google Maps API Web Services) to determine the Canton of a particular University/Institution
4. `folium` to deal with the geographical data and create the Choropleth map

# Building the Choropleth Map

We start off by importing the Python packages mentioned above.

In [1]:
import requests
import folium
import pandas as pd
import numpy as np
import json
import random
from pprint import pprint

## 1. Importing the Data

We downloaded the [Grants Data](http://p3.snf.ch/P3Export/P3_GrantExport.csv) and placed it in the local `data` folder.

In [2]:
GRANTS_FILE = 'data/P3_GrantExport.csv'
grants_data_orig = pd.read_csv(GRANTS_FILE, sep = ';')
grants_data_orig.head(3)

,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN


For our objective, we are interested in the following columns: 

* ** University **: the (possible) university proposing the project. As we see in the third row, a grant can be awarded to an NPO (Non-Profit Organization).
* ** Institution **: the institution proposing the project.
* ** Approved Amount **: amount of approved grants

The Institution/University can be used to deduce the Swiss Canton. Finally, the Amount Approved will be used for our Choropleth map. Let's extract these columns to reduce the size of our DataFrame.

In [3]:
grants_data_red = grants_data_orig[['University', 'Institution', 'Approved Amount']]
grants_data_red.head()

,University,Institution,Approved Amount
0,Nicht zuteilbar - NA,NaN,11619.00
1,Université de Genève - GE,Faculté de Psychologie et des Sciences de l'Ed...,41022.00
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",Kommission für das Corpus philosophorum medii ...,79732.00
3,Universität Basel - BS,Abt. Handschriften und Alte Drucke Bibliothek ...,52627.00
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",Schweiz. Thesauruskommission,120042.00


Finally, we will rename the "Approved Amount" column to "Amount" so that it can be easily accessed.

In [4]:
grants_data_red.rename(columns={'Approved Amount': 'Amount'}, inplace=True)

/home/ubuntu/anaconda2/envs/py3k/lib/python3.5/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


## 2. Cleaning the Data

### Cleaning "Amount"

Let's make sure the "Amount" has the correct variable type, i.e. numeric, and that entries with an invalid "Amount" are dropped.

In [5]:
print(grants_data_red.shape)
grants_clean_amount = grants_data_red
grants_clean_amount.Amount = pd.to_numeric(grants_clean_amount.Amount, errors='coerce')
grants_clean_amount = grants_data_red.dropna(subset=["Amount"])
print(grants_clean_amount.shape)

(63969, 3)
(53059, 3)


/home/ubuntu/anaconda2/envs/py3k/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


We have dropped 10910 entries.

### Cleaning "University" and "Institution"

We only need one of these fields to be valid for each row in order to geolocate where the grant was awarded. So let's drop those rows that have invalid entries for **both** fields. Moreover, we have noticed that the "University" entries have the following structure:

`LONG NAME - SHORT NAME`

Furthermore, one of the entries is `Nicht zuteilbar - NA` which must be SNSF's way of indicating an invalid entry. We will first replace such entries with the standard entry for invalids - `NaN` - before dropping rows.

In [7]:
print(grants_clean_amount.shape)
grants_clean_place = grants_clean_amount.replace(to_replace="Nicht zuteilbar - NA", value=np.nan)
grants_clean_place.dropna(how='all',subset=["University","Institution"], inplace=True)
print(grants_clean_place.shape)

(53059, 3)
(49823, 3)


We have dropped 3236 entries. Almost done! We only one need either "Univerisity" or "Institution" for geolocation. For most entries the "University" should be enough. However, 1437 of the entries are not from Universities but rather Non-Profit Organizations (NPO). This count was determined by running:

`grants_clean.University.value_counts()`

So let's merge the "University" and "Institution" columns into a new column entitled "Institute" and keep the `LONG NAME` of the "University" entry unless it is "NPO (Biblioth., Museen, Verwalt.) - NPO". We also remove anything in the `LONG NAME` that has parenthesis. 

In [8]:
def extract_long_name(entry):
    split_entry = entry.replace("(","-").split("-")
    return split_entry[0]

def set_institute(row): 
    if row["University"] == "NPO (Biblioth., Museen, Verwalt.) - NPO":
        return row["Institution"]
    else: 
        return extract_long_name(str(row["University"]))

grants_clean_place['Institute'] = grants_clean_place.apply(lambda row: set_institute(row), axis=1)
grants_clean_place.head()

,University,Institution,Amount,Institute
1,Université de Genève - GE,Faculté de Psychologie et des Sciences de l'Ed...,41022.0,Université de Genève
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",Kommission für das Corpus philosophorum medii ...,79732.0,Kommission für das Corpus philosophorum medii ...
3,Universität Basel - BS,Abt. Handschriften und Alte Drucke Bibliothek ...,52627.0,Universität Basel
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",Schweiz. Thesauruskommission,120042.0,Schweiz. Thesauruskommission
5,Université de Fribourg - FR,"Séminaire de politique économique, d'économie ...",53009.0,Université de Fribourg


Lastly, we need to drop those cases where "University" as an NPO but the "Institution" was missing/invalid.

In [9]:
print(grants_clean_place.shape)
grants_clean_place = grants_clean_place.dropna(subset=["Institute"])
print(grants_clean_place.shape)

(49823, 4)
(49738, 4)


85 entries were dropped. Now let's extract the "Institute" and the "Amount" and this is what we will use for the rest of the exercise.

In [10]:
grants_clean = grants_clean_place[['Institute', 'Amount']]
grants_clean.head()

,Institute,Amount
1,Université de Genève,41022.0
2,Kommission für das Corpus philosophorum medii ...,79732.0
3,Universität Basel,52627.0
4,Schweiz. Thesauruskommission,120042.0
5,Université de Fribourg,53009.0


### Summary

We have gone from 63696 entries to 49738 by dropping invalid "Amount" and "University" or "Institution" entries. We then created a single "Institute" column as this will be sufficient for geolocation.

## 3. Reading the Canton Data

Now we obtain the abbrevations for all cantons from the geographical JSON data in `data/ch-cantons.topojson.json`. We did some inspection of the structure of the JSON file (using pprint) and extracted the canton abbrevations by navigating appropriately through the JSON data structure.

In [11]:
CH_GEO_JSON = 'data/ch-cantons.topojson.json'

with open(CH_GEO_JSON, 'r') as f:
    canton_data = json.load(f)

cantons = canton_data['objects']['cantons']['geometries']
canton_ids = []
for canton in cantons:
    canton_ids.append(canton['id'])

print(canton_ids)
#pprint(canton_data)

['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO', 'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU']


## 4. Mapping "Institute" to a Canton

To compute grants by canton, we need to find the corresponding Canton for a particular "Institute".

We will first use the **Geonames API** as this was suggested by the assignment but then revert to the results of the **GoogleMaps API** as the latter is able to map more of the "Institutes". 

### Geonames API

In [ ]:
uni_canton = pd.DataFrame(columns=['University','Canton'])

In [ ]:
def canton_in_CH(result):
    for i in range(len(result)):
        if(result['geonames'][i]['countryName']=='Switzerland'):
            return True
    return False

In [ ]:
URL = 'http://api.geonames.org/searchJSON?'

def canton_get(uni):
    paraload = {'q':uni,'username':'shiyuenie'}
    r = requests.get(URL, params=paraload)
    result = r.json()
    return result

In [ ]:
institutes = grants_clean.Institute.unique()

In [ ]:
for ucount in range(len(institutes)): # search full name
    uni = institutes[0][ucount]
    result = canton_get(uni)
    if (result['geonames'] != []):
        if (canton_in_CH(result)):
            uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
        else:
            uni_canton.loc[ucount] = [uni,'Not Found']   
    else:
        uni_canton.loc[ucount] = [uni,'Not Found']
    
uni_canton.head()

In [ ]:
#uni_name=uni_name.replace({r'\s+': '&'}, regex=True)

for ucount in range(len(uni_name)):
    if (uni_canton.loc[:,'Canton'][ucount] == 'Not Found'): # search with seperate parts
        uni_full = uni_name['Full Name'][ucount] 
        uni_short = uni_name['Short Name'][ucount] 
        result = canton_get(uni_full) # search part 1
        if (result['geonames'] != []):
            if(canton_in_CH(result)):
                uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
            else:
                result = canton_get(uni_short) # search part 2 
                if (result['geonames'] != []):
                    if(canton_in_CH(result)):
                        uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
        
uni_canton.head()

In [ ]:
uni_canton[uni_canton['Canton'] != "Not Found"]

for ucount in range(len(uni_test)): # search full name wit &
    uni = uni_test[0][ucount]
    result = canton_get(uni)
    if (result['geonames'] != []):
        if (canton_in_CH(result)):
            uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
    
uni_canton.head()

In [ ]:
np.sum(uni_canton.loc[:,'Canton']=='Not Found')

In [ ]:
uni_canton[uni_canton['Canton'] != "Not Found"]

In [ ]:
np.sum(uni_canton.loc[:,'Canton']!='Not Found')

## GoogleMaps API

We use a Python [API wrapper](https://github.com/googlemaps/google-maps-services-python) for GoogleMaps

In [12]:
# to install the wrapper: !pip install -U googlemaps

    100% |████████████████████████████████| 512kB 2.1MB/s 
  Running setup.py bdist_wheel for googlemaps ... - done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/9a/f9/21/1f17a586dc434abde2935f3d24ecd3b850f46fbff885eac0c6
Successfully built googlemaps
  Found existing installation: requests 2.11.1
    Uninstalling requests-2.11.1:
      Successfully uninstalled requests-2.11.1


In [15]:
import googlemaps

# Key omitted due to security concerns on Github
key = #% env GOOGLE_MAPS_KEY
#########################'

gmaps = googlemaps.Client(key=key)

In [16]:
# We try a sample search to obtain the structure of reply
test_epfl = gmaps.geocode('EPFL Switzerland')
test_epfl

[{'address_components': [{'long_name': 'EPFL',
    'short_name': 'EPFL',
    'types': ['premise']},
   {'long_name': 'Lausanne',
    'short_name': 'Lausanne',
    'types': ['locality', 'political']},
   {'long_name': 'Ouest lausannois',
    'short_name': 'Ouest lausannois',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Vaud',
    'short_name': 'VD',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Switzerland',
    'short_name': 'CH',
    'types': ['country', 'political']},
   {'long_name': '1015', 'short_name': '1015', 'types': ['postal_code']}],
  'formatted_address': 'EPFL, 1015 Lausanne, Switzerland',
  'geometry': {'location': {'lat': 46.5189865, 'lng': 6.5676007},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 46.52033548029151,
     'lng': 6.568949680291502},
    'southwest': {'lat': 46.51763751970851, 'lng': 6.566251719708498}}},
  'place_id': 'ChIJ4zm3ev4wjEcRShTLf2C0UWA',
  'types': ['premise']}

We notice that canton information is returned in 'administrative_area_level_1' field. long_name gives the full name of the canton, short_name gives the abbrevation, the same as the id used in geometry json file.

In [17]:
CANTON_TYPE_KEY = 'administrative_area_level_1'
    
def place_in_CH(place_info):
    for info_dict in place_info['address_components']:
        if info_dict['long_name'] == 'Switzerland':
            return True
    return False

def parse_canton(place_candidates):
    if (place_candidates == None or len(place_candidates) == 0):
        return None
    
    for place in place_candidates:
        if not place_in_CH(place):
            continue
        for info_dict in place['address_components']:
            if (CANTON_TYPE_KEY in info_dict['types']) and (info_dict['short_name'] in canton_ids):
                return info_dict['short_name']
    
    return None    

parse_canton(test_epfl)

'VD'

In [2]:
cnt_found = 0
cnt_unfound = 0
for inst in grants_clean_place.Institute:
    if inst in inst_canton_dic:
        cnt_found += 1
    else:
        cnt_unfound += 1
print(cnt_found)
print(cnt_unfound)

NameError: name 'grants_clean_place' is not defined

In [ ]:
def construct_search_keys(university, institution):
    
    def keys_with(x):
        keys = []
        keys.append(x)
        keys.append(x + ', Switzerland')
        keys.append(x + ', CH')
        return keys
    
    uni_full_name, uni_short_name = university.rsplit('-',1)

    keys = []   
    keys += keys_with(university)
    keys += keys_with(uni_full_name)
    
    if institution != None:
        keys += keys_with(str(institution))
    if uni_short_name != None:
        keys += keys_with(uni_short_name)
    return keys 

In [18]:
def search_canton(keys):
    for key in keys:
        resp = gmaps.geocode(key)
        canton = parse_canton(resp)
        if canton != None:
            return canton
    return None

In [21]:
dic = {}

ddf = grants_clean.copy()

def map_canton(row):
    if row.Institute != None:
        search_keys = []
        search_keys.append(row.Institute)
        search_keys.append(row.Institute + ', Switzerland')
        search_keys.append(row.Institute + ', CH')
        canton = search_canton(search_keys)
        if canton == None:
            print(row.Institute + " None")
        else:
            print(row.Institute + " " + canton)
    return ""

ddf['Canton'] = ddf.apply(map_canton, axis=1)
ddf.head()


Université de Genève  GE
Kommission für das Corpus philosophorum medii aevi der SGG None
Universität Basel  BS
Schweiz. Thesauruskommission None
Université de Fribourg  FR
Université de Fribourg  FR
Universität Zürich  ZH
Université de Lausanne  VD
Université de Genève  GE
Université de Fribourg  FR
Schweizerische Rechtsquellen c/o Universität Zürich / RWI None
Université de Genève  GE
Universität Basel  BS
Universität Zürich  ZH
Université de Fribourg  FR
Université de Genève  GE
Universität Zürich  ZH
Université de Lausanne  VD
Université de Genève  GE
Universität Bern  BE
Universität Zürich  ZH
Université de Genève  GE
Schweizerische Gesellschaft für Volkskunde None
Kuratorium Carl J. Burckhardt None
Universität Zürich  ZH
Eidg. Forschungsanstalt für Wald,Schnee,Land  None
Schweizerisches Nationalmuseum Landesmuseum Zürich ZH
Fondation Hardt pour l'étude de l'Antiquité classique None
Universität Basel  BS
Historische und Antiquarische Gesellschaft zu Basel BS
Université de Genève  G

KeyboardInterrupt: 

In [ ]:
# intialize uni_canton_dict from file
# uni_canton_dict maps university to canton
uni_canton_df = pd.read_csv('uni-canton.csv', encoding='utf-8')
uni_canton_df.set_index('University', inplace=True)
uni_canton_dict = uni_canton_df.to_dict()['0']

In [ ]:
def find_canton(row):
    # if not searched before
    if row.University not in uni_canton_dict:
        search_keys = construct_search_keys(row.University, row.Institution)
        uni_canton_dict[row.University] = search_canton(search_keys)
        print(row.University + " " + uni_canton_dict[row.University])
    return uni_canton_dict[row.University]

In [ ]:
# uni_canton_df = pd.DataFrame.from_dict(uni_canton_dict, orient="index")
# uni_canton_df.index.name = 'University'
# uni_canton_df.to_csv('uni-canton.csv', encoding='utf-8')

In [ ]:
df = data.copy()
df['Canton'] = df.apply(find_canton, axis=1)

In [ ]:
df.head()

In [ ]:
df.to_csv('data-with-canton.csv', encoding='utf-8')

In [ ]:
found_rate = pd.notnull(df['Canton']).sum() / len(df['Canton'])
found_rate

### Manually handle missing data

Fow now, we simply ignore the missing records.

#  Compute grants by canton

In [ ]:
canton_grants_df = df[['Approved Amount', 'Canton']].groupby(['Canton']).mean()
canton_grants_df.reset_index(inplace=True)

In [ ]:
# Add missing data
for canton_id in canton_ids:
    if canton_id not in canton_grants_df['Canton'].values:
        canton_grants_df = canton_grants_df.append({'Canton':canton_id, 'Approved Amount': 0.0}, ignore_index=True)

canton_grants_df = canton_grants_df.sort_values(by=['Approved Amount'], ascending=False)
canton_grants_df

# Data visualization
##  Draw grants map by canton

In [ ]:
map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=canton_grants_df,
               columns=['Canton', 'Approved Amount'], 
               key_on='feature.id', 
               geo_path=CH_GEO_JSON, 
               topojson='objects.cantons', 
               fill_color='YlOrRd'
               )
map

** TODO **: The map cannot be displayed on Github. To view the map, you have to run this cell locally with Jupyter Notebook. To facilitate code reviewing, we may add an additionaly snapshot of the map here.